In [39]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
#import cleaning
import timeit
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer as rt
from nltk.corpus import stopwords
from nltk.corpus import words
# from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle as pkl


In [54]:
import numpy as np

In [40]:
from scipy.sparse import csr_matrix,csc_matrix,hstack

In [41]:
def read_data(fName, colName):
    data = pd.read_csv(fName)
    return data[colName]

# Output result of cleaning into a txt file, by the name of fname
# resultlist is a list of lists, the items in the inner list are strings
def print_result(fName, resultlist):
    with open(fName, 'w') as f:
        for l in result:
            f.write("%s\n" % l)
    f.close()

# Dump the preprocessed data into a pickle file
def generate_cleaned_file(instances, labels):
    col = ["Phrases", "Sentiment"]
    i = range(len((instances)))
    dataframe = pd.DataFrame(columns = col, index = i)
    dataframe["Phrases"] = instances
    dataframe["Sentiment"] = labels
    pkfile = open('preprocessed_train_data.pkl', 'wb')
    rick.dump(dataframe, pkfile)
    pkfile.close()

# Open the pickle file
def open_cleaned_file(fileName):
    pkl = open(fileName, 'rb')
    result = rick.load(pkl)
    pkl.close()
    return result

# Tokenize and filter out non-alphanumeric characters.
def tokenize_data(d):
    # Initialize RegExp tokenizer.
    tokenizer = rt(r'\w+')
    # Make all words lowercase.
    d = (phrase.lower() for phrase in d)
    d = (tokenizer.tokenize(phrase) for phrase in d)
    tokenized = []
    for phrase in d:
        tokenized.append(phrase)
    # print_result('cleaned.txt', result)
    return tokenized

def untokenize(texts):
    docs = []
    for doc in texts:
        temp = ""
        for word in doc:
            temp += word + " "
        docs.append(temp)
    return docs

In [42]:
def filter_data(d):
    # Initialize stopwords list: remove articles, conjunctions, prepositions, pronouns.
    # stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    stop_words = list(stopwords.words('english'))
    # Additional non-sentimental words to filter out
    custom_words = ['genre', 'film', 'movie']
    stop_words = set(stop_words).union(set(custom_words))
    # Second cleaning
    # Initialize dictionary of useful words from nltk corpus
    good_words = set(list(words.words()))
    result = []
    for phrase in d:
        temp = []
        for word in phrase:
            if word in good_words and not word in stop_words:
                temp.append(word)
        result.append(temp)
    # print_result('secondclean.txt', result)
    return result

In [7]:
phrases = read_data("train.csv", "Phrase")
labels = read_data("train.csv", "Sentiment")

In [9]:
cleaned = tokenize_data(phrases)
result = filter_data(cleaned)

In [10]:
vect = CountVectorizer(min_df=2, ngram_range=(0, 30))
X_train = vect.fit(untokenize(result)).transform(untokenize(result))

In [11]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, labels)

/Users/RAJ/anaconda/envs/py7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/RAJ/anaconda/envs/py7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/RAJ/anaconda/envs/py7/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
x_test = read_data("testset_1.csv", "Phrase")

In [34]:
test_phraseid = read_data("testset_1.csv", "PhraseId")

In [52]:
print (type(test_phraseid))

<class 'pandas.core.series.Series'>


In [13]:
cleaned_test = tokenize_data(x_test)
result_test = filter_data(cleaned_test)

In [16]:
X_test = vect.transform((untokenize(result_test)))

In [14]:
lr = grid.best_estimator_

In [17]:
y = lr.predict(X_test)

In [46]:
data = create_df(test_phraseid,y)

In [55]:
df = pd.DataFrame({'PhraseId':np.array(test_phraseid),'Sentiment':np.array(y)})

In [47]:
outfile = 'out.csv'

In [57]:
df.to_csv(outfile,index = False)